In [2]:
#import libraries
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

Using TensorFlow backend.


In [3]:
#define hyperparameters
data_directory = "C:/Users/kashi/Documents/CMPE 258/ind_project/Data2/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json" 
model_weights_directory = 'C:/Users/kashi/Documents/CMPE 258/ind_project/Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [4]:
#generate training model
def make_model(unique_chars):
    
    model = Sequential()
    #Added 3 LSTM layers with dropout 20% and softmax activation in the fully connected layer 
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) 
    
    model.add(Dropout(0.2))
    
    model.add((Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    return model

In [5]:

def generate_sequence(epoch_num, initial_index, seq_length):
    with open(os.path.join(data_directory, charIndex_json)) as f:
        char_to_index = json.load(f)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    unique_chars = len(index_to_char)
    
    model = make_model(unique_chars)
    #load the weights in the weights directory
    model.load_weights(model_weights_directory + "Weights_{}.h5".format(epoch_num))
     
    sequence_index = [initial_index]
    
    for _ in range(seq_length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    cnt = 0
    for i in seq:
        cnt += 1
        if i == "\n":
            break
    seq1 = seq[cnt:]
    
    cnt = 0
    for i in seq1:
        cnt += 1
        if i == "\n" and seq1[cnt] == "\n":
            break
    #ABC sequential model created
    seq2 = seq1[:cnt]
    return seq2

In [6]:
#Results Generated
ep = int(input("1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: "))
ar = int(input("\n2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: "))
ln = int(input("\n3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: "))

music = generate_sequence(ep, ar, ln)

print("\nMUSIC SEQUENCE GENERATED: \n")

print(music)

1. Which epoch number weight you want to load into the model(10, 20, 30, ..., 90). Small number will generate more errors in music: 90

2. Enter any number between 0 to 86 which will be given as initial charcter to model for generating sequence: 25

3. Enter the length of music sequence you want to generate. Typical number is between 300-600. Too small number will generate hardly generate any sequence: 450

MUSIC SEQUENCE GENERATED: 

"D"a3 f3|"A7"A2e "D"f2e|"D7"f2d Afg|"D"a2d d2f||
"A"e2c "E7"Bcd|"A"e2c "E7"Bcd|"A"e3 -"Em"e2d|"A"c2A AGA|"E"B2B "E7"B2e|"A"A3 -"E7"e2d|"A"c2c "B7"acB|"A7"A3 -"D"A2G|"A"A3 -A2:|
K:Am
P:C
e|"Am"e3 e3|"E7"e2d B^cd|"Em"e2e e3|"E7"e2e efa|"A"e2c ABc|
"D"d2B DFG|"Bm"A2F "E7"^GEE|"A"A3 A3|"A"ABc d3|\
"E7"G2B "A"ced|
"A"e2c "E7"Bcd|"A"e3 -"E7"de^d|"A"e2c ABA|
